# LoRA From Scratch – Implement Low-Rank Adaptation for LLMs in PyTorch

In [5]:
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from config import get_lora_config
from datasets import load_dataset

In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [3]:
class LoRALayer(nn.Module):
    """
    Implements the Low-Rank Adaptive Linear Transformation (LoRA) layer.

    This layer performs a linear transformation with learnable low-rank factors,
    reducing computational cost and potentially improving model performance.

    Args:
        fan_in: Number of input features (int).
        fan_out: Number of output features (int).
        rank: Rank of the low-rank factors (int).
        alpha: Hyperparameter scaling the output (int).

    Attributes:
        alpha: Hyperparameter scaling the output (int).
        A: Low-rank factor matrix of shape (fan_in, rank) (float tensor).
        B: Low-rank factor matrix of shape (rank, fan_out) (float tensor).

    Inputs:
        x: Input tensor of shape (..., fan_in) (float tensor).

    Outputs:
        Transformed tensor of shape (..., fan_out) (float tensor).
    """

    def __init__(self, fan_in: int, fan_out: int, rank: int, alpha: int):
        super().__init__()
        self.alpha = alpha
        std = torch.tensor(rank, dtype=torch.float) ** -0.5
        self.A = nn.Parameter(torch.randn(fan_in, rank) * std)
        self.B = nn.Parameter(torch.zeros(rank, fan_out))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.alpha * (x @ self.A @ self.B)

In [4]:
linear_layer = nn.Linear(10_000, 10_000)
lora_layer = LoRALayer(10_000, 10_000, 8, 4)
count_parameters(linear_layer), count_parameters(lora_layer) # (10_000 * 10_000 + 10_000), (10_000 * 8 * 2)

(100010000, 160000)

In [5]:
x = torch.randn(50, 10_000)
out_linear = linear_layer(x)
out_lora = lora_layer(x)
out_linear.shape, out_lora.shape # the same shape (50, 10_000)

(torch.Size([50, 10000]), torch.Size([50, 10000]))

In [6]:
class LinearWithLoRA(nn.Module):
    """
    Combines a linear layer with a LoRALayer to perform a combined transformation.

    This module wraps a linear layer and adds a LoRALayer in parallel. The
    output is the sum of the linear layer's output and the LoRALayer's output.

    Args:
        linear_layer: The original linear layer to be wrapped (nn.Module).
        rank: Rank of the low-rank factors in the LoRALayer (int).
        alpha: Hyperparameter scaling the LoRALayer output (int).

    Attributes:
        linear_layer: Original linear layer (frozen, nn.Module).
        alpha: Hyperparameter scaling the LoRALayer output (int).
        lora_layer: LoRALayer instance (nn.Module).

    Inputs:
        x: Input tensor of shape (..., in_features) (float tensor).

    Outputs:
        Transformed tensor of shape (..., out_features) (float tensor).
    """

    def __init__(self, linear_layer: nn.Module, rank: int, alpha: int):
        super().__init__()
        self.linear_layer = linear_layer
        self.alpha = alpha
        self.lora_layer = LoRALayer(linear_layer.in_features, linear_layer.out_features, rank, alpha)

    def forward(self, x):
        return self.linear_layer(x) + self.lora_layer(x)

In [11]:
print(count_trainable_parameters(linear_layer))
linear_with_lora = LinearWithLoRA(linear_layer, 8, 4)
linear_layer.requires_grad_(False)
count_trainable_parameters(linear_with_lora)

100010000


160000

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
print(model)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [9]:
def lora_model(model: nn.Module, config: dict = None, **kwargs) -> nn.Module:
    """
    Applies LoRA (Low-Rank Adaptive Linear Transformation) to a pre-trained model.

    This function selectively replaces linear layers within a model with
    corresponding LoRALayer instances based on a configuration dictionary.
    The configuration specifies which parts of the model to apply LoRA to
    and hyperparameters like rank and alpha.

    Args:
        model: The pre-trained model to be modified (nn.Module).
        config: Optional dictionary containing LoRA configuration (default: None).
                If not provided, a default configuration is retrieved using
                `get_lora_config` (kwargs are passed to this function).
        **kwargs: Additional keyword arguments passed to `get_lora_config`.

    Returns:
        The modified model with LoRA layers applied (nn.Module).
    """

    # Ensure config is available
    if config is None:
        config = get_lora_config(**kwargs)

    # Freeze the pre-trained model parameters
    model.requires_grad_(False)

    def assign_lora(linear_layer):
        return LinearWithLoRA(linear_layer, rank=config["lora_rank"], alpha=config["lora_alpha"])

    for layer in model.distilbert.transformer.layer:
        if config['lora_query']:
            layer.attention.q_lin = assign_lora(layer.attention.q_lin)
        if config['lora_key']:
            layer.attention.k_lin = assign_lora(layer.attention.k_lin)
        if config['lora_value']:
            layer.attention.v_lin = assign_lora(layer.attention.v_lin)
        if config['lora_projection']:
            layer.attention.out_lin = assign_lora(layer.attention.out_lin)
        if config['lora_ffn']:
            layer.ffn.lin1 = assign_lora(layer.ffn.lin1)
            layer.ffn.lin2 = assign_lora(layer.ffn.lin2)

    if config['lora_head']:
        model.pre_classifier = assign_lora(model.pre_classifier)
        model.classifier = assign_lora(model.classifier)

    return model

In [10]:
print(count_parameters(model))
print(count_trainable_parameters(model))

model = lora_model(model)

print(count_parameters(model))
print(count_trainable_parameters(model))

66955010
66955010
67637010
682000


In [4]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

c:\Users\wwww\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\wwww\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
dataset_name = "dair-ai/emotion"

dataset = load_dataset(dataset_name, )

print(f"Dataset name: {dataset_name}")
print(f"Available splits: {list(dataset.keys())}")

train_data = dataset["train"]

print(f"Sample data point: {train_data[0]}")


Using the latest cached version of the module from C:\Users\wwww\.cache\huggingface\modules\datasets_modules\datasets\dair-ai--emotion\cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd (last modified on Wed Mar 13 22:45:14 2024) since it couldn't be found locally at dair-ai/emotion, or remotely on the Hugging Face Hub.


Dataset name: dair-ai/emotion
Available splits: ['train', 'validation', 'test']
Sample data point: {'text': 'i didnt feel humiliated', 'label': 0}


In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [10]:
len(train_data)

16000

In [11]:
def tokenize_text(batch):
    return 

In [12]:
batch = {'text': 'i did not feel humiliated', 'label': 0}
out = tokenize_text(batch)

In [13]:
out

{'input_ids': [101, 1045, 2106, 2025, 2514, 26608, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [14]:
import torch
from torch.utils.data import Dataset


class EmotionDataset(Dataset):
    def __init__(self, dataset, tokenizer, partition_key="train") -> None:
        super().__init__()
        self.partition = dataset[partition_key]
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.partition)

    def __getitem__(self, index):
        text = self.partition[index]['text']
        label = torch.tensor(self.partition[index]['label'], dtype=torch.int8)

        tokenized = self.tokenizer(text, truncation=True, padding=True)
        input_ids = torch.tensor(tokenized['input_ids'], dtype=torch.int64)
        attention_mask = torch.tensor(tokenized['attention_mask'], dtype=torch.int8)

        return {
            'text': text,
            'label': label,
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }


In [15]:
train_dataset = EmotionDataset(dataset, tokenizer, 'train')

In [17]:
train_dataset[0]

{'text': 'i didnt feel humiliated',
 'label': tensor(0, dtype=torch.int8),
 'input_ids': tensor([  101,  1045,  2134,  2102,  2514, 26608,   102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1], dtype=torch.int8)}